## Core imports

In [ ]:
import numpy as np
import xarray as xr

## Sample Data for Unit Testing

## Wet Bulb Zero Height (Snow Level Proxy)

In [ ]:
def calc_wbzh(tw, gh, orog):
    
    ''' 
    Function to calculate the height of the wet bulb zero 
    (or other chosen wet bulb temperature)
    
    Input arrays are assumed as xarray with minimum of numeric 'level' coordinate
    
    Parameters:
        tw : array_like
            1-D (vertical) or n-D array of Wet Bulb Temperature (C)

        gh : array_like
            1-D (vertical) or n-D array of geopotential height (m)
            Note if model provides true geopotential divide by g=9.80665 
            to obtain the geopotential height in meters

        orog : array_like
            Scalar or n-D array of surface elevation (m)
            Suggest using model orography at native resolution or
            DEM for upsampled/downscaled grids

    Returns:
        wbzh : n-D array or scalar
            The height of the wet bulb zero parameter in meters
            This is a scalar if tw and gh are 1-D arrays
    
    University of Utah
    Michael Wessler, Trevor Alcott
    '''
    
    # WBZ parameter: temperature to search for 
    # 0.5 degC as in Western Region tech. attachment
    wbzparam = 0.5
    
    for i, level in enumerate(tw.level.values):
        
        if i > 0:

            level_top = tw.isel(level=i).level.values
            level_bot = tw.isel(level=i-1).level.values
            print('Searching for WBZ between %d and %d hPa'%(level_bot, level_top))

            gh_top = gh.isel(level=i)
            gh_bot = gh.isel(level=i-1)

            tw_top = tw.isel(level=i)
            tw_bot = tw.isel(level=i-1)

            # Linear interpolation of wbz height to use when/if it is between these two levels
            interp_wbzh = gh_bot + ((wbzparam - tw_bot)*((gh_top - gh_bot)/(tw_top - tw_bot)))

            if i == 1:
                # First iteration, establish the wbz height (wbzh) array
                # If WBZ between these two levels, use interpolated WBZH, else np.nan
                wbzh = xr.where( (tw_bot >= wbzparam) & (tw_top <= wbzparam), interp_wbzh, np.nan)

            else:
                # If does not exist:
                wbzh = xr.where( ((tw_bot >= wbzparam) & (tw_top <= wbzparam)) & (np.isnan(wbzh)), interp_wbzh, wbzh)

                # If exists and wbzh subterrainian
                wbzh = xr.where( ((tw_bot >= wbzparam) & (tw_top <= wbzparam)) & (~np.isnan(wbzh) & (wbzh >= _orog.min())), interp_wbzh, wbzh)

    # Where nans remain because entire column Tw < wbzparam, fill with 0 m AMSL
    wbzh = xr.where(np.isnan(wbzh) & (_tw.max(dim='level') < wbzparam), 0, wbzh)
    
    return wbzh

## Layer temperature calculation

In [ ]:
def calc_tlayer(t, gh, orog):
    
    ''' 
    Function to calculate the relevant layer temperature 
    as predictor for curve fit snow-liquid-ratio calculation
    based on Alcott and Steenburgh (2010) Stepwise Multiple
    Linear Regression based SLR algorithim
    
    Input arrays are assumed as xarray with minimum of numeric 'level' coordinate
    
    Parameters:
        t : array_like
            1-D (vertical) or n-D array of Wet Bulb Temperature (C)

        gh : array_like
            1-D (vertical) or n-D array of geopotential height (m)
            Note if model provides true geopotential divide by g=9.80665 
            to obtain the geopotential height in meters

        orog : array_like
            Scalar or n-D array of surface elevation (m)
            Suggest using model orography at native resolution or
            DEM for upsampled/downscaled grids

    Returns:
        tlayer : n-D array or scalar
            Relevant layer temperature (predictor for SLR curve fit)
            This is a scalar if t and gh are 1-D arrays
    
    University of Utah
    Michael Wessler, Trevor Alcott
    '''
    
    # Determine geopotential height relative to ground level
    # + 500 m buffer (see Alcott(?), I believe this may have been done as a bias correction)
    gh_agl = (gh - (orog + 500.0))
    
    # Where this is 0.0 m, set to 1.0 m
    gh_agl = xr.where(gh_agl == 0.0, 1.0, gh_agl)
    
    # If the 1000mb height is > 0, use the 1000 mb temperature to start
    # Otherwise assign t=0
    tvals = xr.where(gh_agl.sel(level=1000) > 0, t.sel(level=1000), 0)
    
    # Iterate through the vertical levels
    for i in range(t.level.size)[:0:-1]:
        
        # 'l' level
        # 'z' geopotential height
        # 'c/up/dn' current level/level above/level below
        
        # Current level
        lc = t.level.isel(level=i).values
        zc = gh_agl.isel(level=i)
        tc = t.isel(level=i)
        
        # Level above (corrected for 'wraparound' when iterating)
        up = i+1 if i+1 < t.level.size else 0
        lup = t.level.isel(level=up).values
        zup = gh_agl.isel(level=up)
        tup = t.isel(level=up)
        
        # Level below (corrected for 'wraparound' when iterating)
        ldn = t.level.isel(level=i-1).values
        zdn = gh_agl.isel(level=i-1)
        tdn = t.isel(level=i-1)
        
        # Print values for a sanity check while testing 
        # to ensure proper iteration/vertical wrap
        # print(i, lc, lup, ldn)
        
        # Where the geopotential height AGL is > 0 at this level 
        # and geopotential height AGL is < 0 at level below...
        tvals = xr.where(((zc > 0.0) & (zdn < 0.0)),
        
        # Determine a layer temperature
        (( zc / ( zc - zup ) ) * ( tup - tc ) + tc ),
        
        # Else use layer temperature already determined
        tvals)
    
    # In the strange exception case where 500 mb is below ground level
    # apply T500 as Tlayer (redundant failsafe - probably not needed)
    tlayer = xr.where(gh_agl.sel(level=500) < 0, t.sel(level=500), tvals)
        
    return tlayer

## Snow-Liquid-Ratio Calculation

In [ ]:
def calc_slr(tlayer, wbzh, orog):
    
    ''' 
    Function to calculate the relevant layer temperature 
    as predictor for curve fit snow-liquid-ratio calculation
    based on Alcott and Steenburgh (2010) Stepwise Multiple
    Linear Regression based SLR algorithim
    
    Input arrays are assumed as xarray with minimum of numeric 'level' coordinate
    
    Parameters:
        tlayer : array_like
            1-D (vertical) or n-D array of relevant layer temperature (C)
            predictor for SLR curve fit

        wbzh : array_like
            1-D (vertical) or n-D array of the height of 
            the wet bulb zero parameter (m)

        orog : array_like
            Scalar or n-D array of surface elevation (m)
            Suggest using model orography at native resolution or
            DEM for upsampled/downscaled grids

    Returns:
        slr : n-D array or scalar
            Snow-Liqud-Ratio value interger (e.g. 15 for 15:1)
            This is a scalar if tlayer and wbzh are 1-D arrays
    
    University of Utah
    Michael Wessler, Trevor Alcott
    '''
    
    # Tunable transition layer parameters (m)
    all_snow_buffer = 0
    transition_layer = 200
    
    # Extend the snow level below the wet bulb zero parameter height if set
    snow_level = wbzh - all_snow_buffer
    snow_level = xr.where(snow_level < 0., 0., snow_level)

    # Curve fit to Alcott and Steenburgh (2010) SMLR results
    init_slr = xr.where(tlayer < 0., 5. - tlayer, 5.)
    init_slr = xr.where(tlayer < -15., 20. + (tlayer + 15.), init_slr)
    init_slr = xr.where(tlayer < -20., 15., init_slr)

    # Keep the initial SLR calculations above the snow level
    slr = xr.where(orog >= snow_level, init_slr, 0.)

    # Linear attenuation of the SLR in the transition layer
    slr = xr.where(
        ((orog < snow_level) & (orog > (snow_level - transition_layer))),
        (init_slr * (orog - (snow_level - transition_layer)) / transition_layer), slr)

    return slr

In [ ]:
wbzh = calc_wbzh(tw, gh, orog)
tlayer = calc_tlayer(t, gh, orog)
slr = calc_slr(tlayer, wbzh, orog)